<html>
<head>
  <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;500;700&display=swap" rel="stylesheet">
  <style>
    body {
      background-color: #f5f5f5; /* Light background */
      color: #333; /* Dark text for readability */
      padding: 40px;
      font-family: 'Roboto', sans-serif;
      text-align: left;
    }
    .container {
      width: 600px;
      margin: 20px 0;
    }
    .icon {
      margin-right: 10px;
      vertical-align: middle;
    }
    h1, h2 {
      color: #0077b6; /* Primary color for headings */
    }
    p, span {
      color: #555; /* Subtle text color */
    }
    .spacer {
      margin-top: 40px; /* Custom spacer for larger gaps */
    }
  </style>
</head>
<body>

  <!-- Title Section -->
  <div class="container">
    <h1 style="margin: 0; font-size: 36px; letter-spacing: 1.5px;">
      <img src="https://img.icons8.com/ios-filled/50/0077b6/brain.png" width="40" class="icon">AI-Fall 03-CA2
    </h1>
  </div>

  <!-- University Section -->
  <div class="container">
    <h2 style="font-size: 24px; margin-bottom: 10px;">
      <img src="https://img.icons8.com/ios-filled/50/0077b6/university.png" width="30" class="icon">University
    </h2>
    <div style="display: flex; align-items: center;">
      <img src="https://upload.wikimedia.org/wikipedia/en/thumb/f/fd/University_of_Tehran_logo.svg/225px-University_of_Tehran_logo.svg.png" width="60px" style="margin-right: 10px;">
      <span style="font-size: 18px;">University of Tehran</span>
    </div>
  </div>

  <!-- Custom Spacer to increase gap between University and Student Info -->
  <!-- <div class="spacer"></div> -->

  <!-- Student Info Section -->
  <div class="container">
    <h2 style="font-size: 20px; margin-bottom: 10px;">
      <img src="https://img.icons8.com/ios-filled/50/0077b6/student-male.png" width="30" class="icon">Student Info
    </h2>
    <span style="font-size: 18px;">Amirhossein Arefzadeh - std id: 810101604</span>
  </div>

  <!-- Description Section -->
  <div class="container">
    <h2 style="font-size: 24px; margin-bottom: 10px;">
      <img src="https://img.icons8.com/ios-filled/50/0077b6/info.png" width="30" class="icon">Project Overview
    </h2>
    <p style="font-size: 18px; line-height: 1.6;">
    This project explores the application of two distinct algorithmic approaches (genetic algorithms and minimax algorithms) in the context of game strategy and image generation. The goal is to understand and analyze their effectiveness, performance, and adaptability in solving complex problems within these domains.
    </p>
  </div>
  
</body>
</html>


# Genetic

In [11]:
import random
from PIL import Image, ImageDraw
import numpy as np
import cv2
import os
from copy import deepcopy

In [12]:
OFFSET = 10

def generate_point(width, height):
    x = random.randrange(0 - OFFSET, width + OFFSET, 1)
    y = random.randrange(0 - OFFSET, height + OFFSET, 1)
    return (x, y)

class Triangle:
    def __init__(self, img_width, img_height):
        self.points = []
        for i in range(3):
            self.points.append(generate_point(img_width,img_height))


        self.color = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
        )

        self._img_width = img_width
        self._img_height = img_height

    def duplicate(self):
        t = Triangle(self._img_width, self._img_height)
        t.color = self.color
        t.points = self.points.copy()
        return t

In [13]:
class Chromosome:
    def __init__(self,img_height, img_width,target_image,num_triangles):
        self.img_height = img_height
        self.img_width = img_width
        self.background_color = (0,0,0,255)
        self.triangles = []
        self.target_image = target_image
        self.num_trinagles = num_triangles
        for _ in range(self.num_trinagles):
            triangle = Triangle(self.img_width, self.img_height)
            self.triangles.append(triangle)

    def draw(self) -> Image:
        size = self.target_image.size
        img = Image.new('RGB', size, self.background_color)
        draw = Image.new('RGBA', size)
        pdraw = ImageDraw.Draw(draw)
        for triangle in self.triangles:
            colour = triangle.color
            points = triangle.points
            pdraw.polygon(points, fill=colour, outline=colour)
            img.paste(draw, mask=draw)
        return img

## mutation

In [14]:
def mutate_all_fiture(self, mutation_rate=1, swap_rate=0.25):
    mutations = ['shift', 'point', 'color', 'reset']
    weights = [25, 30, 40, 5]
    triangle = random.choice(self.triangles)

    mutation_type = random.choices(mutations, weights=weights, k=1)[0]
    if mutation_type == 'shift':
        x_shift = random.randint(-10, 10)
        y_shift = random.randint(-10, 10)
        triangle.points = [(x + x_shift, y + y_shift) for x, y in triangle.points]
    elif mutation_type == 'point':
        for j in range(3):
          new_x = min(self.img_width, max(0, triangle.points[j][0] + random.randint(-20, 20)))
          new_y = min(self.img_height, max(0, triangle.points[j][1] + random.randint(-20, 20)))
          triangle.points[j] = (new_x , new_y)
    elif mutation_type == 'color':
        triangle.color = tuple(
            max(0, min(255, c + random.randint(-20, 20))) for c in triangle.color
        )
    elif mutation_type == 'reset':
        new_triangle = Triangle(self.img_width, self.img_height)
        triangle.points = new_triangle.points
        triangle.color = new_triangle.color

Chromosome.mutate_all_fiture = mutate_all_fiture

## Fitness Evaluation

### Mean Squared Error (MSE) Fitness

The `mse_fitness` function evaluates the fitness of a generated image by calculating the Mean Squared Error (MSE) between it and the target image. MSE is a commonly used metric for measuring the average of the squared differences between corresponding pixel values, making it sensitive to large differences and ideal for penalizing substantial deviations from the target.

In the function:
1. Both the generated and target images are converted to arrays to allow element-wise operations.

2. The MSE value, `diff`, is computed as the mean of the squared differences between corresponding pixels:
   $$
   \text{MSE} = \frac{1}{N} \sum_{i=1}^{N} \left( (R_{\text{target},i} - R_{\text{created},i})^2 + (G_{\text{target},i} - G_{\text{created},i})^2 + (B_{\text{target},i} - B_{\text{created},i})^2 \right)
   $$
   where \( N \) is the total number of pixels in the image.
3. A lower MSE value indicates a closer resemblance to the target image, as it implies smaller differences on average between the pixel values of the generated and target images.

The MSE metric is particularly useful in scenarios where significant deviations from the target image are heavily penalized, driving the genetic algorithm to refine the image closer to the target at each iteration.


In [15]:
def mse_fitness(self) -> float:
    created_image = np.array(self.draw())
    target_image_array = np.array(self.target_image)
    diff = np.mean((created_image - target_image_array) ** 2)
    return diff

Chromosome.mse_fitness = mse_fitness

### Difference Fitness

The `diff_fitness` function evaluates the fitness of a generated image by measuring the absolute pixel-wise difference between it and the target image. This method calculates the sum of absolute differences for each pixel, providing a straightforward metric for similarity. Unlike the Delta E fitness method, which accounts for human perception in color differences, this approach directly compares the RGB values of each pixel.

In the function:
1. The generated image and the target image are converted to arrays for pixel-wise operations.

2. The fitness value, `diff`, is calculated as the sum of the absolute differences between corresponding pixel values:
   $$
   \text{diff} = \sum |R_{\text{target}} - R_{\text{created}}| + |G_{\text{target}} - G_{\text{created}}| + |B_{\text{target}} - B_{\text{created}}|
   $$
3. A lower `diff` score signifies a closer match between the generated and target images, guiding the genetic algorithm toward producing images that better resemble the target.

This fitness function is simpler and computationally less intensive than Delta E, making it suitable for cases where speed is prioritized over color accuracy in visual similarity.


In [16]:
def diff_fitness(self) -> float:
    created_image = np.array(self.draw())
    target_image_array = np.array(self.target_image)
    # Absolute difference
    diff = np.sum(np.abs(created_image - target_image_array))
    return diff

Chromosome.diff_fitness = diff_fitness

### Delta E Fitness

The `deltaE_fitness` function calculates the fitness of a generated image by measuring its color similarity to a target image. This similarity is assessed using the Delta E metric, a color difference measurement designed to approximate human visual perception. Developed by the International Commission on Illumination, Delta E serves as a standard metric to quantify color discrepancies in the LAB color space, which represents colors more uniformly in relation to human vision.

In the function:
1. The generated image and the target image are first converted to the LAB color space as `np.float32` arrays for accuracy.

2. The Delta E calculation is performed as the Euclidean distance between corresponding pixels in the two images:
   $$
   \Delta E = \sqrt{\sum \left( (L_{\text{target}} - L_{\text{generated}})^2 + (a_{\text{target}} - a_{\text{generated}})^2 + (b_{\text{target}} - b_{\text{generated}})^2 \right)}
   $$
3. The fitness score is the mean of all Delta E values across pixels, giving an average color difference for the entire image.

A lower fitness score indicates a closer resemblance to the target image, guiding the genetic algorithm to evolve towards a more accurate representation.


In [17]:
def deltaE_fitness(self):
    created_image = np.array(self.draw()).astype(np.float32)
    target_image_array = np.array(self.target_image).astype(np.float32)

    # Calculate delta E as Euclidean distance in LAB color space
    delta_e = np.sqrt(np.sum((target_image_array - created_image) ** 2, axis=-1))
    self.fitness = np.mean(delta_e)
    return self.fitness

Chromosome.deltaE_fitness = deltaE_fitness

In [18]:
class GeneticAlgorithm():
    def __init__(self,max_width,max_height,target_image, population_size, triangles_number):
        self.population_size = population_size
        self.max_width = max_width
        self.max_height = max_height
        self.population = [Chromosome(max_height,max_width,target_image, triangles_number) for i in range(population_size)]
        self.target_image = target_image
        self.num_triangles = triangles_number
        
        # Initialize lists to store fitness data for plotting
        self.best_fitness = []
        self.average_fitness = []
        self.worst_fitness = []

    def calc_fitnesses(self):
        fitnesses = []
        for chromosome in self.population:
            fitnesses.append(chromosome.deltaE_fitness())
        return fitnesses

    def sort_population(self, fitnesses):
        return [x for _, x in sorted(zip(fitnesses, self.population), key=lambda pair: pair[0])]

    def mutation(self,next_gen):
      for chromosome in next_gen:
        chromosome.mutate_color_points()

    def run(self, n_generations):
        save_dir = "generation_images"
        os.makedirs(save_dir, exist_ok=True)

        for iteration in range(n_generations):
            fitnesses = self.calc_fitnesses()

            # Calculate fitness metrics
            best_fitness_val = np.min(fitnesses)
            avg_fitness_val = np.mean(fitnesses)
            worst_fitness_val = np.max(fitnesses)

            # Store fitness metrics for plotting
            self.best_fitness.append(best_fitness_val)
            self.average_fitness.append(avg_fitness_val)
            self.worst_fitness.append(worst_fitness_val)

            if iteration % 10 == 0:
                fit_arr = np.array(fitnesses)
                print(f"Fitness in Generation {iteration}: mean: {np.mean(fit_arr)}, max: {np.max(fit_arr)}, min: {np.min(fit_arr)}")

                if iteration % 100 == 0:
                    best_chromosome = self.sort_population(fitnesses)[0]
                    best_image = best_chromosome.draw()
                    best_image.save(os.path.join(save_dir, f"gen_{iteration}.png"))

            self.sel_from_best(fitnesses)

    def get_best_of_population(self):
        fitnesses = self.calc_fitnesses()
        best_population = self.sort_population(fitnesses)
        image = best_population.draw()
        cv2.imshow("Reconstructed Image", np.array(image))
        cv2.waitKey(0)
        cv2.destroyAllWindows()

## crossover

In [19]:
def uniform_cross_over(self, parent1, parent2):
    child1 = Chromosome(self.max_height, self.max_width, self.target_image, self.num_triangles)
    child2 = Chromosome(self.max_height, self.max_width, self.target_image, self.num_triangles)

    for i in range(self.num_triangles):
        if random.random() < 0.5:
            child1.triangles[i] = parent1.triangles[i].duplicate()
            child2.triangles[i] = parent2.triangles[i].duplicate()
        else:
            child1.triangles[i] = parent2.triangles[i].duplicate()
            child2.triangles[i] = parent1.triangles[i].duplicate()

    return child1, child2

GeneticAlgorithm.uniform_cross_over = uniform_cross_over

## next generation selection policy

In [20]:
def sel_from_best(self, fitnesses):
    sorted_population = self.sort_population(fitnesses)
    next_gen = []
    num_elites = int(0.1 * self.population_size)
    elites = sorted_population[:num_elites]
    next_gen.extend(elites)

    num_parents = int(0.5 * self.population_size)
    parents = sorted_population[:num_parents]
    while len(next_gen) < self.population_size:
        parent1, parent2 = random.sample(parents, 2)
        child1, child2 = self.uniform_cross_over(deepcopy(parent1), deepcopy(parent2))
        child1.mutate_all_fiture()
        child2.mutate_all_fiture()

        next_gen.extend([deepcopy(child1), deepcopy(child2)])

    self.population = next_gen[:self.population_size]

GeneticAlgorithm.sel_from_best = sel_from_best

In [ ]:
def resize(image,max_size):
    new_width = int((max_size/max(image.size[0],image.size[1]))* image.size[0])
    new_height = int((max_size/max(image.size[0],image.size[1]))* image.size[1])
    image = image.resize((new_width,new_height), resample=Image.Resampling.LANCZOS)  
    return image

In [22]:
target_image_path = "C:/Users/Amir/Desktop/دانشگاه/AI/CAs/Genetic-and-Minimax-Algorithms/target_images/eagle.jpg"
image = Image.open(target_image_path)
image = resize(image,100)

width,height = image.size
population_size = 50
triangles_number = 50
alg = GeneticAlgorithm(width,height,image, population_size,triangles_number)
alg.run(50000)

Fitness in Generation 0: mean: 178.67877197265625, max: 208.6144561767578, min: 144.43826293945312
Fitness in Generation 10: mean: 109.0727310180664, max: 126.69812774658203, min: 103.11990356445312
Fitness in Generation 20: mean: 95.86227416992188, max: 98.90979766845703, min: 92.69426727294922
Fitness in Generation 30: mean: 88.98365020751953, max: 93.39473724365234, min: 87.50267028808594
Fitness in Generation 40: mean: 85.33154296875, max: 92.26350402832031, min: 82.55371856689453
Fitness in Generation 50: mean: 80.16703033447266, max: 81.68569946289062, min: 78.51544189453125
Fitness in Generation 60: mean: 75.8558349609375, max: 78.18144226074219, min: 74.35496520996094
Fitness in Generation 70: mean: 71.62366485595703, max: 81.78675079345703, min: 69.37044525146484
Fitness in Generation 80: mean: 68.39187622070312, max: 71.30868530273438, min: 67.34822082519531
Fitness in Generation 90: mean: 63.368953704833984, max: 71.06590270996094, min: 62.45960235595703
Fitness in Generatio

KeyboardInterrupt: 

## Game

In [ ]:
import numpy as np
import random
import pygame
import math
from time import sleep, time

ROW_COUNT = 6
COLUMN_COUNT = 7
SQUARESIZE = 100
RADIUS = int(SQUARESIZE / 2 - 5)
PLAYER = 1
CPU = -1
EMPTY = 0
PLAYER_PIECE = 1
CPU_PIECE = -1
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
WINDOW_LENGTH = 4
WINING_SCORE = 10000000000000

In [ ]:
class Connect4UI:
    def __init__(self, width=COLUMN_COUNT*SQUARESIZE, height=(ROW_COUNT+1)*SQUARESIZE):
        pygame.init()
        self.width = width
        self.height = height
        self.size = (self.width, self.height)
        self.screen = pygame.display.set_mode(self.size)
        self.font = pygame.font.SysFont("monospace", 75)

    def draw_board(self, board):
        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT):
                pygame.draw.rect(self.screen, BLUE, (c * SQUARESIZE, r * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
                pygame.draw.circle(self.screen, BLACK, (int(c * SQUARESIZE + SQUARESIZE / 2), int(r * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)

        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT):
                if board[r][c] == PLAYER_PIECE:
                    pygame.draw.circle(self.screen, RED, (int(c * SQUARESIZE + SQUARESIZE / 2), self.height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
                elif board[r][c] == CPU_PIECE:
                    pygame.draw.circle(self.screen, YELLOW, (int(c * SQUARESIZE + SQUARESIZE / 2), self.height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)

        pygame.display.update()
        sleep(0.2)
        

    def display_winner(self, winner):
        if winner == PLAYER:
            label = self.font.render("Player wins!!", 1, RED)
        elif winner == CPU:
            label = self.font.render("Computer wins!!", 1, YELLOW)
        else:
            label = self.font.render("It's a draw!!", 1, BLUE)
        self.screen.blit(label, (40, 10))
        pygame.display.update()
        sleep(5)

In [ ]:
class Connect4Game:
    def __init__(self, ui, minimax_depth, prune):
        self.board = np.zeros((ROW_COUNT, COLUMN_COUNT))
        self.ui = Connect4UI() if ui else None
        self.minimax_depth = minimax_depth
        self.prune = prune
        self.current_turn = random.choice([1, -1])
    
    # toy moshakhasat mohre ra garar midahad   
    def drop_piece(self, board, row, col, piece):
        board[row][col] = piece

    #radif khaly bady dar an soton ra midahad
    def get_next_open_row(self, board,col):
        for r in range(ROW_COUNT):
            if board[r][col] == 0:
                return r
    #
    def print_board(self, board):
        print(np.flip(board, 0))

    def winning_move(self, board, piece):
        for c in range(COLUMN_COUNT - 3):
            for r in range(ROW_COUNT):
                if all(board[r][c+i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT - 3):
                if all(board[r+i][c] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COLUMN_COUNT - 3):
            for r in range(ROW_COUNT - 3):
                if all(board[r+i][c+i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COLUMN_COUNT - 3):
            for r in range(3, ROW_COUNT):
                if all(board[r-i][c+i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        return False
    
    def evaluate_window(self, window, piece):
        score = 0
        opp_piece = PLAYER_PIECE
        if piece == PLAYER_PIECE:
            opp_piece = CPU_PIECE
        
        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(EMPTY) == 1:
            score += 5
        elif window.count(piece) == 2 and window.count(EMPTY) == 2:
            score += 2
        if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
            score -= 4
        
        return score
    
    def score_position(self, board, piece):
        score = 0  
        center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
        center_count = center_array.count(piece)
        score += center_count * 3
    
        for r in range(ROW_COUNT):
            row_array = [int(i) for i in list(board[r,:])]
            for c in range(COLUMN_COUNT-3):
                window = row_array[c:c+WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        for c in range(COLUMN_COUNT):
            col_array = [int(i) for i in list(board[:,c])]
            for r in range(ROW_COUNT-3):
                window = col_array[r:r+WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        for r in range(ROW_COUNT-3):
            for c in range(COLUMN_COUNT-3):
                window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
                score += self.evaluate_window(window, piece)
                
        for r in range(ROW_COUNT-3):
            for c in range(COLUMN_COUNT-3):
                window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
                score += self.evaluate_window(window, piece)
    
        return score
    
    #aya bazi be payan reside ya hich harekaty bagi namonde
    def is_terminal_node(self, board):
        return self.winning_move(board, PLAYER_PIECE) or self.winning_move(board, CPU_PIECE) or len(self.get_valid_locations(board)) == 0
    
    #tamam location hayi ke mitavan mohre dakhelesh andakh ra midahad
    def get_valid_locations(self, board):
        valid_locations = []
        for col in range(COLUMN_COUNT):
            if board[ROW_COUNT-1][col] == 0:
                valid_locations.append(col)
        return valid_locations
    
    def best_cpu_score(self, board, moves):
        move = None
        max_score = -math.inf
        for col in moves:
            row = self.get_next_open_row(board, col)
            b_copy = board.copy()
            self.drop_piece(b_copy, row, col, CPU_PIECE)
            score = self.heuristic(b_copy, CPU_PIECE)
            if score > max_score:
                max_score = score
                move = col
        return move
    
    #emtyaz dehi be terminal va non_terminal nodes
    def heuristic(self, board, piece):
        if(self.is_terminal_node(board)):
            if self.winning_move(board, piece):
                return WINING_SCORE
            elif self.winning_move(board, -piece):
                return -WINING_SCORE
            else:
                return 0
        else:
            return self.score_position(board, piece) - self.score_position(board, -piece)
        
    #TODO: Implement minimax algorithm with alpha-beta pruning and depth limiting
    #self.prune is a boolean that indicates whether to use alpha-beta pruning or not
    #Use the heuristic function as the evaluation function for non terminal nodes
    #Return the column and the score of the best move
    def minimax(self, board, depth, alpha, beta, player):
        valid_locations = self.get_valid_locations(board)
        is_terminal = self.is_terminal_node(board)
        if depth == 0 or is_terminal:
            if is_terminal:
                return None, WINING_SCORE if player == CPU_PIECE else -WINING_SCORE
            return None, self.heuristic(board, player)

        if player == PLAYER_PIECE:
            max_score = -math.inf
            best_col = random.choice(valid_locations)

            for col in valid_locations:
                row = self.get_next_open_row(board, col)
                b_copy = board.copy()
                self.drop_piece(b_copy, row, col, PLAYER_PIECE)
                score = self.minimax(b_copy, depth - 1, alpha, beta, CPU_PIECE)[1]
                if score > max_score:
                    max_score = score
                    best_col = col
                alpha = max(alpha, score)
                if beta <= alpha and self.prune:
                    break  # Alpha-beta pruning
            return best_col, max_score
        
        else:
            min_score = math.inf
            best_col = random.choice(valid_locations)

            for col in valid_locations:
                row = self.get_next_open_row(board, col)
                b_copy = board.copy()
                self.drop_piece(b_copy, row, col, CPU_PIECE)
                score = self.minimax(b_copy, depth - 1, alpha, beta, PLAYER_PIECE)[1]
                if score < min_score:
                    min_score = score
                    best_col = col
                beta = min(beta, score)
                if beta <= alpha and self.prune:
                    break  # Alpha-beta pruning
            return best_col, min_score
        
    def get_cpu_move(self, board, randomness_percent=30):
        moves = self.get_valid_locations(board)
        if len(moves) == 0:
            return None
        random_move = random.choice(moves)
        score_move = self.best_cpu_score(board, moves)
        move = random.choice([score_move] * (100 - randomness_percent) + [random_move] * randomness_percent)
        return move, self.get_next_open_row(board, move)


    def get_human_move(self, board, depth = 1):
        col = self.minimax(board, depth, -math.inf, math.inf, self.current_turn)[0]
        return col, self.get_next_open_row(board, col)

    def play(self):
        winner = None
        while not self.is_terminal_node(self.board):
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == PLAYER:
                col, row = self.get_human_move(self.board, self.minimax_depth)
                if col is not None:
                    self.drop_piece(self.board, row, col, PLAYER_PIECE)
                    if self.winning_move(self.board, PLAYER_PIECE):
                        winner = PLAYER
                    self.current_turn = CPU
            else:
                col, row = self.get_cpu_move(self.board)
                if col is not None:
                    self.drop_piece(self.board, row, col, CPU_PIECE)
                    if self.winning_move(self.board, CPU_PIECE):
                        winner = CPU
                    self.current_turn = PLAYER
            
            if self.ui:
                self.ui.draw_board(self.board)
                if winner is not None:
                    self.ui.display_winner(winner)
                            
        if winner is None:
            winner = 0
        return winner

Complete the Connect4Game class below by implementing the `minimax algorithm` with `alpha-beta pruning` and `depth limiting`. Don't change other parts of the code unless you want to define a new heuristic function.

In [ ]:
def print_results(results, depth, pruning, start, end):
    pruning_status = "Enabled" if pruning else "Disabled"
    print(
        f'Depth: {depth} | Pruning: {pruning_status} -> User Wins: {results[1]:3}, CPU Wins: {results[-1]:2}, Ties: {results[0]:2}, Time: {end - start:.2f}s'
    )

Use the code below to test the game with the UI. Pygame is not compatible with Jupyter Notebook and will not work in this environment. To use Pygame, please run the code in a separate `.py` file.

In [ ]:
# game = Connect4Game(True, 3, True)
# game.play()

Run the following code to test the results of the game and the time taken to play the game in different `depths` and with `pruning` enabled and disabled.

In [ ]:
def check_results():
    for pruning in [True, False]:  
        for d in range(1, 4):
            results = {-1: 0, 0: 0, 1: 0}
            start = time()
            for _ in range(20):  
                game = Connect4Game(False, d, pruning)  
                results[game.play()] += 1
            end = time()
            print_results(results, d, pruning, start, end)

check_results()

Depth: 1 | Pruning: Enabled -> User Wins:   0, CPU Wins: 20, Ties:  0, Time: 0.74s
Depth: 2 | Pruning: Enabled -> User Wins:  19, CPU Wins:  0, Ties:  1, Time: 3.48s
Depth: 3 | Pruning: Enabled -> User Wins:  12, CPU Wins:  8, Ties:  0, Time: 5.37s
Depth: 1 | Pruning: Disabled -> User Wins:   0, CPU Wins: 20, Ties:  0, Time: 0.72s
Depth: 2 | Pruning: Disabled -> User Wins:  20, CPU Wins:  0, Ties:  0, Time: 4.26s
Depth: 3 | Pruning: Disabled -> User Wins:  19, CPU Wins:  1, Ties:  0, Time: 21.27s
